In [ ]:
# Import some libraries

import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision.utils import save_image
from torchvision import transforms
from torch.autograd import Variable
from torchvision.datasets import FashionMNIST
from matplotlib import pyplot as plt

In [ ]:
# Convert vector to image

def to_img(x):
    x = 0.5 * (x + 1)
    x = x.view(x.size(0), 28, 28)
    return x

In [ ]:
# Displaying routine

def display_images(in_, out, n=1):
    for N in range(n):
        if in_ is not None:
            in_pic = to_img(in_.cpu().data)
            plt.figure(figsize=(18, 6))
            for i in range(4):
                plt.subplot(1,4,i+1)
                plt.imshow(in_pic[i+4*N])
                plt.axis('off')
        out_pic = to_img(out.cpu().data)
        plt.figure(figsize=(18, 6))
        for i in range(4):
            plt.subplot(1,4,i+1)
            plt.imshow(out_pic[i+4*N])
            plt.axis('off')

In [ ]:
# Define data loading step

batch_size = 128

img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset = FashionMNIST('./data', transform=img_transform, download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

testset = FashionMNIST('./data/', transform=img_transform)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=False)


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# Define model architecture 

class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            #1st Layer
            nn.Conv2d(in_channels=1,out_channels=16, kernel_size=3, stride=3, padding=1),#output of size 10
            nn.LeakyReLU(),
            nn.AvgPool2d(kernel_size=2, stride=2, padding=0),#output of size 5
            #2nd layer
            nn.Conv2d(in_channels=16, out_channels=8, kernel_size=3, stride=2, padding=1), #output of size 3
            nn.LeakyReLU(),
            nn.AvgPool2d(kernel_size=2, stride=1, padding=0), #output of size 2
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(in_channels=8, out_channels=16, kernel_size=3, stride=2, padding=0),
            nn.LeakyReLU(),
            nn.ConvTranspose2d(in_channels=16, out_channels=8, kernel_size=5, stride=3, padding=1),
            nn.LeakyReLU(),
            nn.ConvTranspose2d(in_channels=8, out_channels=1, kernel_size=2, stride=2, padding=1),
            nn.Tanh(),
        )
        #Since we're interested in classification we can ignore 
        #the decoding part of the last network and just feed the output of 
        #the deepest hidden layer into the classifier layer: the output of the encoding part(size 2)

        self.fc = nn.Sequential(
            nn.Linear(8*2*2, 100),
            nn.ReLU(),
            nn.Linear(100, 200),
            nn.ReLU(),
            nn.Linear(200, 300),
            nn.ReLU(),
            nn.Linear(300, 10),
        )
        
    

    def forward(self, x):
        x = self.encoder(x)
        #x = self.decoder(x)
        x = x.view(-1,8*2*2 )
        x = self.fc(x)
        x = nn.functional.log_softmax(x,dim=1)
        return x
    
model = Autoencoder().to(device)
criterion = nn.CrossEntropyLoss()   #Since we are doing classification, we use the Cross Entropy function

In [ ]:
# Configure the optimiser
learning_rate = 1e-3
l2_regularizer = 1e-5
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=learning_rate,
    weight_decay=l2_regularizer,
)

In [ ]:
num_epochs = 5
loss_list = list()
for epoch in range(num_epochs):  # loop over the dataset multiple times
    for i, data in enumerate(dataloader, 0):
        # get the inputs
        real, labels = data
        
        inputs = Variable(real)
        labels = Variable(labels).long() 
        
        # zero the parameter gradients
        optimizer.zero_grad()             
        
        # forward + backward + optimize
        outputs = model(inputs)           #forward pass
        loss = criterion(outputs, labels) # compute the loss
        loss.backward()                   #compute the gradients of the loss:backward pass
        optimizer.step()                  #update the parameters
        
        #print statistics
    print('epoch [{}/{}], loss:{:.4f}'
      .format(epoch+1, num_epochs, loss.data[0]))
    loss_list.append(loss.data[0])
    #print(loss_list)
    #if epoch % 1 == 0:
    #    pic = to_img(outputs.data)
    #    save_image(pic, './AE_img/{}.png'.format(epoch))

#torch.save(model.state_dict(), './conv_autoencoder.pth') 
print('Finished Training')


# Plot of the loss of training
nb_epoch = range(1, len(loss_list) + 1)
plt.plot(nb_epoch, loss_list)
plt.xlabel('Epochs ',fontsize=9)
plt.ylabel('Loss',fontsize=9)
plt.title('Plot of the Training Loss',fontsize=16)
plt.show()

In [ ]:
#Test of our network with a test set
correct = 0    #count of the number of correct predictions
total = 0      #count the total number of predictions

with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = Variable(images)  #get the images in the mini-batch
        labels = Variable(labels)  #get the labels of images in the mini-batch
        outputs = model(images)    #do prediction of each image in the mini-batch
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += float((predicted == labels).sum().item())

print('Accuracy of the network: %f %%' % (
    100 * correct / total))

In [ ]:
#Lets print out the accuracy of the network on each class
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = Variable(images)
        labels = Variable(labels)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of classe %5s : %2f %%' % (
        [i+1], 100 * class_correct[i] / class_total[i]))
